## ARDR

## import environment

In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt  
from sklearn.preprocessing import minmax_scale
%matplotlib inline
from scipy import stats
from datetime import datetime
import os

from numpy import mean
from numpy import std
from numpy import absolute
from pandas import read_csv

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error


import itertools 
from time import time
from functools import reduce 
import operator

def timer(start_time=None):
    if not start_time:
        start_time = datetime.now()
        return start_time
    elif start_time:
        thour, temp_sec = divmod((datetime.now() - start_time).total_seconds(), 3600)
        tmin, tsec = divmod(temp_sec, 60)
        print('\n Time taken: %i hours %i minutes and %s seconds.' % (thour, tmin, round(tsec, 2)))

#Seaborn
import seaborn as sns #Visualization
plt.rcParams['figure.figsize'] = [8,5]
plt.rcParams['font.size'] =14
plt.rcParams['font.weight']= 'bold'
plt.style.use('seaborn-whitegrid')

## Import model

In [2]:
from sklearn.linear_model import ARDRegression # Import model

# Reading Data

In [3]:
trainDf = pd.read_csv ("data/ML-CUP20-TR.csv")

# Normalize
colnames=trainDf.columns.values
trainDf=pd.DataFrame(minmax_scale(trainDf, feature_range=(0,1), axis=0))
trainDf.columns=colnames
trainDf.describe()

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,Y1,Y2
count,1524.000000,1524.000000,1524.000000,1524.000000,1524.000000,1524.000000,1524.000000,1524.000000,1524.000000,1524.000000,1524.000000,1524.000000
mean,0.493986,0.455558,0.422902,0.532771,0.542641,0.556990,0.448495,0.479733,0.477938,0.494039,0.425603,0.351685
std,0.229482,0.218699,0.211772,0.174884,0.209266,0.168283,0.216930,0.206977,0.214500,0.224177,0.287916,0.271691
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.277706,0.261214,0.236137,0.407235,0.353918,0.448039,0.244393,0.305093,0.281794,0.286610,0.164375,0.130400
50%,0.515967,0.464428,0.416107,0.527146,0.576631,0.533931,0.475399,0.480954,0.482409,0.536919,0.366169,0.296888
75%,0.686938,0.638466,0.584685,0.648318,0.715638,0.665114,0.635441,0.637042,0.659932,0.668569,0.672862,0.502768
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


## Full dataset

In [4]:
dataset=trainDf.values
Xtot = dataset[:, 0:10]
Ytot = dataset[:, 10:12]
ytot = dataset[:, 10]

## Split dataset 60%

In [5]:
first_split = trainDf.sample(frac=0.6,random_state=200)
second_split=trainDf.drop(first_split.index)
trainset = first_split.to_numpy()
testset = second_split.to_numpy()


In [6]:
X = trainset[:, 0:10]
Y = trainset[:, 10:12]  # Y1 and Y2 train
y = trainset[:, 10]     # Y1 train
yb = trainset[:, 11]    # Y2 

Xtest= dataset[:, 0:10]
Ytest = dataset[:, 10:12] # Y1 and Y2 test
ytest= dataset[:, 10]     # Y1 test
ybtest= dataset[:, 11]    # Y2 test

## MODEL DATA NEED TO CHANGE!

In [7]:
scoring='neg_mean_squared_error'
n_jobs=1
modelname="ARDR"

In [8]:
## Dont change this tab

In [9]:
gridsy1=[]
gridsy2=[]
result_list=[]

grid_dict = {}

gridindexcounter=0
def safe_arange(start, stop, step):
    return step * np.arange(start / step, stop / step)

In [10]:

grid_params = [{ 
            'base_estimator__n_iter': [50,100,150],
            'base_estimator__alpha_1': safe_arange(0.1,1,0.1),
            'base_estimator__alpha_2': safe_arange(0.1,1,0.1),
            'base_estimator__lambda_1': safe_arange(0.1,1,0.1),
            'base_estimator__lambda_2': safe_arange(0.1,1,0.1)}]

print(grid_params)
pipe= Pipeline([('scl', StandardScaler()), ('base_estimator', ARDRegression())])
		
gsY1 = GridSearchCV(estimator=pipe,
			param_grid=grid_params,
            n_jobs=n_jobs,
			scoring=scoring,
			cv=10)
gsY2 = GridSearchCV(estimator=pipe,
			param_grid=grid_params,
            n_jobs=n_jobs,
			scoring=scoring,
			cv=10)
gridsy1.append(gsY1)  
gridsy2.append(gsY2)  
grid_dict[gridindexcounter]=modelname
gridindexcounter+=1

[{'base_estimator__n_iter': [50, 100, 150], 'base_estimator__alpha_1': array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]), 'base_estimator__alpha_2': array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]), 'base_estimator__lambda_1': array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]), 'base_estimator__lambda_2': array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])}]


In [ ]:
print('Performing model optimizations...')
best_acc = 0.0
best_clf = 0
best_gs = ''

for idx, gs in enumerate(gridsy1):
    start_time = timer(None) # timing starts from this point for "start_time" variable
    print('\nEstimator: %s' % grid_dict[idx])	
    # Fit grid search	
    grid_result=gs.fit(X, y)
    result_list.append(grid_result)
    timer(start_time) # timing ends here for "start_time" variable
    # Best params
    print('Best params: %s' % gs.best_params_)
    # Best training data accuracy
    print('Best training accuracy: %.3f' % gs.best_score_)
    # Predict on test data with best params
    y_pred = gs.predict(Xtest)

    # Test data accuracy of model with best params
    print('Test set accuracy score for best params: %.3f ' % mean_squared_error(ytest, y_pred))
    # Track best (highest test accuracy) model

    if mean_squared_error(ytest, y_pred) > best_acc:
        best_acc = mean_squared_error(ytest, y_pred)
        best_gs = gs
        best_clf = idx
        
best_acc = 0.0
best_clf = 0
best_gs = ''

for idx, gs in enumerate(gridsy2):
    start_time = timer(None) # timing starts from this point for "start_time" variable
    print('\nEstimator: %s' % grid_dict[idx])	
    # Fit grid search	
    grid_result=gs.fit(X, yb)
    result_list.append(grid_result)
    timer(start_time) # timing ends here for "start_time" variable
    # Best params
    print('Best params: %s' % gs.best_params_)
    # Best training data accuracy
    print('Best training accuracy: %.3f' % gs.best_score_)
    # Predict on test data with best params
    y_predb = gs.predict(Xtest)

    # Test data accuracy of model with best params
    print('Test set accuracy score for best params: %.3f ' % mean_squared_error(ybtest, y_predb))
    # Track best (highest test accuracy) model

    if mean_squared_error(ybtest, y_predb) > best_acc:
        best_acc = mean_squared_error(ybtest, y_predb)
        best_gs = gs
        best_clf = idx

Performing model optimizations...

Estimator: ARDR


In [ ]:
plt.scatter(ytest,ybtest)
plt.scatter(y_pred,y_predb)